In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 1000)

In [5]:
df = pd.read_csv("/home/jone/NLP/Dataset/train_category.csv")
df.head()

,text,label
0,앵커 애틀랜타 총격 사건 으로 미국시민 들 은 또한 번 거리 로 나왔습니다 슬프고 ...,international
1,앵커 이번 사건 에서 희생 된한 인명 의 신원 이 사흘 만에 공개 됐습니다 유가족 ...,international
2,일오 전시 부터 버팀목 자금 을 통해 짝수 소 상공 인들 로부터 버팀목 자금 신청 ...,economy
3,일 오후 서울 황학동 중앙시장 중 고 가전제품 판매점 에 진열 된에서 문재인 대통령...,economy
4,실종 주 째 고양시 발달장애 인 점퍼 한 강서 발견 경기 고양시 행주산성 둘레길 에...,society


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

label_encoder = Tokenizer()
label_encoder.fit_on_texts(df['label'])
label = np.array(label_encoder.texts_to_sequences(df['label'])) - 1
category = label_encoder.word_index
print(category)

{'society': 1, 'politics': 2, 'international': 3, 'economy': 4, 'sport': 5, 'entertain': 6, 'it': 7, 'culture': 8}


In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'])
vocab = tokenizer.word_index
vocab_size = len(vocab) + 1
tfidfm = tokenizer.texts_to_matrix(df['text'], mode='tfidf')
print(tfidfm.shape)

(2196, 53371)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(tfidfm, label, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(1756, 53371) (440, 53371) (1756, 1) (440, 1)


In [19]:
model_cls = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(vocab_size,)),
    tf.keras.layers.Dense(len(category), activation='softmax')
])

model_cls.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
model_cls.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               6831616   
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 1032      
Total params: 6,832,648
Trainable params: 6,832,648
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model_cls.fit(X_train, Y_train, epochs=10, batch_size=128, validation_data=(X_test, Y_test))

Epoch 1/10
14/14 [==============================] - 3s 236ms/step - loss: 5.3188e-04 - accuracy: 1.0000 - val_loss: 0.5053 - val_accuracy: 0.8818
Epoch 2/10
14/14 [==============================] - 1s 94ms/step - loss: 4.7248e-04 - accuracy: 1.0000 - val_loss: 0.5051 - val_accuracy: 0.8841
Epoch 3/10
14/14 [==============================] - 1s 92ms/step - loss: 4.2095e-04 - accuracy: 1.0000 - val_loss: 0.5047 - val_accuracy: 0.8841
Epoch 4/10
14/14 [==============================] - 1s 106ms/step - loss: 3.7492e-04 - accuracy: 1.0000 - val_loss: 0.5047 - val_accuracy: 0.8864
Epoch 5/10
14/14 [==============================] - 1s 85ms/step - loss: 3.3487e-04 - accuracy: 1.0000 - val_loss: 0.5047 - val_accuracy: 0.8864
Epoch 6/10
14/14 [==============================] - 1s 82ms/step - loss: 3.0484e-04 - accuracy: 1.0000 - val_loss: 0.5048 - val_accuracy: 0.8864
Epoch 7/10
14/14 [==============================] - 1s 97ms/step - loss: 2.7124e-04 - accuracy: 1.0000 - val_loss: 0.5049 - val_

In [12]:
df_test = pd.read_csv("/Users/zum/Dev/Dataset/test_category.csv")
df_test.head()

,text,label
0,후배 여배우 성폭행 의혹 에 휘 말린 배우 배 진웅 측 이 허위 사실 유포 에 대한...,entertain
1,조이 뉴스 정명화 기자 배우 배 진웅 이 동료 여배우 성추행 혐의 를 부인 했다 배...,entertain
2,일 서 울 영등포구 국민은행 여의도 지점 스마트 딜링룸 전광판 에 코스피지수 가 전...,economy
3,정부 가 일 신종 코로나바이러스 감염증 코로나 확산 에 따른 의료 현 장인 력 부족...,society
4,정부 가일 부터 코로나 로 피해 를 입은소 상공 인 과 고용 취약 계층 에 버팀목 ...,society


In [13]:
test_tfidfm = tokenizer.texts_to_matrix(df_test['text'], mode='tfidf')
test_label = np.array(label_encoder.texts_to_sequences(df_test['label']))
print(test_tfidfm.shape, test_label.shape)

(942, 53371) (942, 1)


In [14]:
model_cls.evaluate(test_tfidfm, test_label)

30/30 [==============================] - 0s 16ms/step - loss: 0.4923 - accuracy: 0.8588


[0.4922669529914856, 0.85881108045578]

In [16]:
pred = model_cls.predict(test_tfidfm).argmax(axis=1)
df_test['pred'] = label_encoder.sequences_to_texts(pred.reshape(-1, 1))
print(df_test[df_test['label'] != df_test['pred']])

30/30 [==============================] - 0s 14ms/step
                                                  text          label           pred
12   온라인 서 이재용 옥중 특별 회견 문 일파 만 파 삼성 전혀 사실 아냐 변호인 통해...        economy        society
34   영화 미나리 정이 삭 감독 이 미국 감독 조합 이 수여 하 는 감독 상 후보 에 올...        culture  international
35   이준석 전 국민 의 힘 최고 위원 은 직장인 익명 커뮤니티 블라인드 에 부러우면 이...       politics        economy
36   화성시 가 봄철 미세먼지 계절 관리제 기간 을 맞아이 달말까지 농촌 지역 폐기물 불...       politics        society
41   경 기 파주시 는 지난 일 부터 오는 월말 까지 일정 으로 부동산 거래 가격 거짓 ...        society        economy
46   게임 업계 의 연 봉인 상 바람 이 이어지고있는가운 데크 래 프톤 도 개발 직 군 ...             it        economy
49   앙겔라 메르켈 독일 총리 가 지난 일 현 지 시간 백신 생산 업체 및 주 총리 들 ...        economy  international
53   공 수 처 장 지 명일 에도 참모 들 에 확실히 해결 하 라 문재인 대통령 이 지난...       politics        society
55   우리나라 의 유기발광다이오드 디스플레이 가 올해 수출 억 달러 를 목표 로 하고있다...        economy             it
56   산업 통상 자원부 는 성윤 모 장관 이일오 전시 서울 마 곡동 디스플레이 오프라인 ...        economy             it
66   입국 후 코